<a href="https://colab.research.google.com/github/nncliff/qwen-32B/blob/main/chapter-1/ipynb/kvCache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KV Cache（キー・バリューキャッシュ）の実装

このノートブックでは、大規模言語モデル（LLM）推論の効率化において重要な最適化技術である**KV Cache**メカニズムを解説します。

**KV Cacheとは？**
自己回帰生成（GPTのような）では、モデルは一度に1つのトークンを生成します。キャッシュがない場合、モデルは*すべての過去のトークン*に対してアテンションのキーとバリューを毎ステップ再計算する必要があります。KV Cacheはこれらの事前計算されたキーとバリューを保存し、モデルは*最新の*トークンに対してのみ計算すればよくなります。

**このノートブックの内容：**
1.  キャッシュをサポートする`SimpleDecoderBlock`
2.  キャッシュの更新と削除を管理する`KVCacheManager`
3.  キャッシュがどのように使用・更新されるかを示すマルチターン会話のシミュレーション

### `k`と`v`には何が格納されているのか？

**FFNの結果が格納されているのか？**
**このコードでは：はい。**
`forward`メソッドを見てみましょう：
```python
x = self.ffn(self.norm2(x))
if use_cache:
    return x, (x.clone(), x.clone())
```
このコードは`x`（FFN/ブロックの出力）をキャッシュに保存しています。

**3. これは標準的なTransformerの動作なのか？**
**いいえ、これは簡略化されたデモンストレーションです。**
実際のTransformer（GPTやLlamaなど）では：
*   KV Cacheは Attention層内部の**キーとバリューの射影**を格納します。
*   FFNの出力は格納**しません**。
*   ブロックの出力は格納**しません**。

*注：「本物の」KV cacheを実装するには、`nn.MultiheadAttention`を使用せず、カスタムAttention層を書く必要があります。PyTorchのモジュールは内部のK/V射影を隠蔽しているためです。*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import string
from typing import List, Tuple

In [ ]:
class SimpleDecoderBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int):
        super(SimpleDecoderBlock, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.ReLU(),
            nn.Linear(4 * embed_dim, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x: torch.Tensor, kv_cache = None, use_cache: bool = True) -> torch.Tensor:
        # x: (batch_size, seq_len, embed_dim)

        if kv_cache is not None:
            # Use cached key and value tensors for efficient decoding
            k, v = kv_cache
            x_attn, _ = self.self_attn(self.norm1(x), k, v, need_weights=False)
        else:
            # Compute self-attention normally
            x_attn, _ = self.self_attn(self.norm1(x), self.norm1(x), self.norm1(x), need_weights=False)

        x = x + x_attn
        x = self.ffn(self.norm2(x))

        if use_cache:
            # Update kv_cache with new key and value tensors
            return x, (x.clone().detach(), x.clone().detach())

        return x, None

### `forward`におけるKV Cacheの動作

`if kv_cache is not None:`ブロックが最適化の核心です。

1.  **最初の呼び出し（プリフィル）**：モデルが初めてプロンプトを見るとき、`kv_cache`は通常`None`です。モデルはすべてのプロンプトトークンを並列に処理します（`else`ブロック）。
2.  **後続の呼び出し（デコーディング）**：新しいトークンを生成するとき（または会話の新しいターンを処理するとき）、`x`として**新しいトークンのみ**を渡します。
    *   履歴全体のアテンションを再計算する代わりに、事前計算されたキーとバリューの行列を`kv_cache`経由で提供します。
    *   モデルは新しい`x`（クエリ）を`kv_cache`内の履歴（キー/バリュー）に対してアテンドします。

**別のバッチなのか？**
通常、いいえ。推論では「バッチサイズ」は並列に生成している独立したシーケンスの数を指します。
*   `x`は*現在の*バッチの**新しいタイムステップ**を表します。
*   `kv_cache`は*現在の*バッチの**過去のタイムステップ**を表します。

全く関連のないリクエスト（別のユーザー）を処理する場合は、確かに空のキャッシュから始めます。

### 例：チャットボットの会話

「トークン生成」と「新しいターン」を明確にするため、会話で視覚化してみましょう。

**1. ターン1（ユーザーが「Hi」と言う）→ プリフィルフェーズ**
*   **入力（`x`）：** "Hi"（プロンプト全体）
*   **キャッシュ：** 空（`None`）
*   **動作：** モデルは"Hi"をゼロから処理。
*   **結果：**
    *   最初のトークンを予測："Hel"
    *   キャッシュを返す：KV("Hi")

**2. 応答生成（AIが「Hello」を続ける）→ デコーディングフェーズ**
*   **入力（`x`）：** "Hel"（**新しく生成された**トークンのみ）
*   **キャッシュ：** KV("Hi")（ステップ1から）
*   **動作：** モデルは"Hel"をキャッシュされた"Hi"に対してアテンド。
*   **結果：**
    *   次のトークンを予測："lo"
    *   キャッシュを返す：KV("Hi", "Hel")

**3. ターン2（ユーザーが「How are you?」と言う）→ 新しいターンのプリフィル**
*   **コンテキスト：** 履歴は"Hi"（ユーザー）+ "Hello"（AI）。
*   **入力（`x`）：** "How are you?"（新しいユーザー入力）
*   **キャッシュ：** KV("Hi", "Hello")（前のターンから保存）
*   **動作：**
    *   "Hi"と"Hello"を再処理**しません**。
    *   `x="How are you?"`と`kv_cache=KV("Hi", "Hello")`を渡します。
    *   モデルは"How are you?"のアテンションを計算し、キャッシュされた"Hi"と"Hello"にアテンドします。

このノートブックのループでは、`round_id=2`はまさに**ターン2**と同じです。新しいトークン（`token_tensors`）を入力しますが、履歴（`kv_cache`）を提供することで、モデルは再計算せずにコンテキストを理解できます。

In [ ]:
class KVCacheManager:
    def __init__(self, max_cache_size: int = 64):
        self.cache : List[Tuple[torch.Tensor, torch.Tensor]] = []
        self.token_labels : List[str] = [] # To store labels for each token in the cache
        self.max_cache_size = max_cache_size

    def get_cache(self) -> Tuple[torch.Tensor, torch.Tensor]:
        if not self.cache:
            return None

        k = torch.cat([item[0] for item in self.cache], dim=1)  # Concatenate along sequence length
        v = torch.cat([item[1] for item in self.cache], dim=1)  # Concatenate along sequence length
        return (k, v) # shape of k or v: (batch_size, total_sequence_length, embed_dim)

    def update_cache(self, new_kv: Tuple[torch.Tensor, torch.Tensor], tokens : List[str], current_round : int):
        self.cache.append(new_kv)
        self.token_labels += [f"Round{current_round}"] * new_kv[0].size(1)  # Assuming new_kv[0] shape is (batch_size, seq_len, embed_dim)

        if len(self.token_labels) > self.max_cache_size:
            # Keep only current round tokens if cache is full
            # Note: The original logic was trying to filter based on labels.
            # Since we append new_kv (current round) at the end, and we want to keep "Round{current_round}",
            # we can simply keep the last element of the cache if we assume previous rounds are what we want to discard.

            # Simplified logic to avoid tensor unpacking errors from original code
            self.cache = [self.cache[-1]]
            self.token_labels = [label for label in self.token_labels if label == f"Round{current_round}"]

In [ ]:
def generate_tokens(prompt : str, vocab : List[str], num_tokens: int = 5) -> List[str]:
    return [random.choice(vocab) for _ in range(num_tokens)]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
decoder = SimpleDecoderBlock(embed_dim=64, num_heads=4).to(device)
kv_cache_manager = KVCacheManager(max_cache_size=30)
vocab = list(string.ascii_lowercase)  # Example vocabulary

for round_id in range(1, 6):
    prompt = f"[Round {round_id}] User Input: write an function"
    tokens = generate_tokens(prompt, vocab)
    print(f"Round {round_id} generated tokens: {' '.join(tokens)}")

    # Simulate token embeddings
    token_tensors = torch.stack([torch.randn(64) for _ in tokens]).unsqueeze(0).to(device)  # shape: (1, seq_len, embed_dim)

    # Retrieve kv_cache and decode
    kv_cache = kv_cache_manager.get_cache()
    output, new_kv = decoder(token_tensors, kv_cache=kv_cache, use_cache=True)

    if new_kv is not None:
        kv_cache_manager.update_cache(new_kv, tokens, current_round=round_id)

    summary = ''.join(random.choices(string.ascii_lowercase, k=10))
    print(f"Round {round_id} summary: {summary}")

print("\n=== Final KV Cache State ===")
print(f"Current token number in cache: {len(kv_cache_manager.token_labels)}")
print(f"Round labels in cache: {kv_cache_manager.token_labels}")

Using device: cpu
Round 1 generated tokens: b o r s f
Round 1 summary: jbdnwhasmr
Round 2 generated tokens: o q x z a
Round 2 summary: rmutdasaeh
Round 3 generated tokens: g j y x t
Round 3 summary: jbxluxwkrr
Round 4 generated tokens: v k v f m
Round 4 summary: eaiafvpwcw
Round 5 generated tokens: d h b x d
Round 5 summary: mvyfrugpva

=== Final KV Cache State ===
Current token number in cache: 25
Round labels in cache: ['Round1', 'Round1', 'Round1', 'Round1', 'Round1', 'Round2', 'Round2', 'Round2', 'Round2', 'Round2', 'Round3', 'Round3', 'Round3', 'Round3', 'Round3', 'Round4', 'Round4', 'Round4', 'Round4', 'Round4', 'Round5', 'Round5', 'Round5', 'Round5', 'Round5']
